# Term frequency and inverse document frequency

In this final Notebook looking at how to handle textual data, we will consider how *inverse document frequency* is used to weight terms in the term frequency vectors. We will use the same training and test documents as in [the previous Notebook](22.3 Applying the classifier to a real dataset.ipynb) so that we can compare the performance of the two techniques directly.

In the module material we discuss the technique of inverse document frequency weighting as well as stopword removal. Although we will not look at stopword removal in the Notebooks, while working through this Notebook you should think about how different techniques can be used to improve the performance of your data investigations. When working on your own investigation, you should always be thinking about how you would go about selecting different ways of treating the data.

## Initial imports and function definitions

In [ ]:
# Standard imports
import pandas as pd

import math, string
import os 

from scipy.spatial.distance import cosine
from sklearn.neighbors import KNeighborsClassifier

from collections import Counter

We will use the same definitions for the main functions as in [Notebook 22.3](22.3 Applying the classifier to a real dataset.ipynb). In this case, we will use `tokenise_email_document` again, rather than the simpler `tokenise_document`.

In [ ]:
def tokenise_email_document(emailDocIn_txt):
    '''Convert an input string to a list of tokens using the operations:
    
        - convert to lower case
        - split on whitespace
        - remove surrounding punctuation
    '''
    return [token.strip(string.punctuation)  # remove punctuation around tokens
            
            for token in emailDocIn_txt.lower().split()] # Convert to lower case and split
                                                         # on whitespace 

In [ ]:
def build_term_index(tokenisedDocuments_coll):
    '''Return a set of all the terms appearing in the 
       documents in tokenisedDocuments_coll
    '''
    allTerms_set = set()  # Store the tokens as a set to remove repetitions
    
    for tokens_coll in tokenisedDocuments_coll:
        allTerms_set = allTerms_set.union(set(tokens_coll))
        
    return list(allTerms_set)     # Return the members as a list

In [ ]:
def build_tf_vector(tokenisedDocument_ls, termIndex_ls):
    '''Return a pandas Series representing the term 
       frequency vector of the tokenised document 
       tokenisedDocument_ls, and indexed with termIndex_ls
    '''
    
    return pd.Series(Counter(tokenisedDocument_ls),
                     index=termIndex_ls).fillna(0)

## Import data

We also need to import the same training and test data as for the previous Notebook.

To recap, we are using 1000 ham documents and 1000 spam documents as training data.

The ham training data can be found in the folder:

    data/trainingData/ham/
    
and the spam training data in the folder:

    data/trainingData/spam/
   

We have also selected 200 ham documents and 200 spam documents to use as test data.

The ham test data can be found in the folder:

    data/testData/ham/
    
and the spam test data in the folder:

    data/testData/spam/

In [ ]:
trainingCorpusDocuments_ls = []
trainingCorpusClasses_ls = []

# First collect the ham documents:
print("Reading ham training files...")

for (path, dirs, files) in os.walk('./data/trainingData/ham/'):
    
    for file in files:
        if file[0] == '.':  # Don't process hidden files
            continue
        
        with open(os.path.join(path, file), 'rb') as fileIn:
            docText = fileIn.read()
            docText = docText.decode('utf-8', 'ignore')   # decoding the utf-8
            
            trainingCorpusDocuments_ls.append(docText)
            trainingCorpusClasses_ls.append('ham')

# Next, collect the spam documents:
print("Reading spam training files...")

for (path, dirs, files) in os.walk('./data/trainingData/spam/'):
    
    for file in files:
        if file[0] == '.':  # Don't process hidden files
            pass
        else:
            with open(os.path.join(path, file), 'rb') as fileIn:
                docText = fileIn.read()
                docText = docText.decode('utf-8', 'ignore')   # decoding the utf-8

                trainingCorpusDocuments_ls.append(docText)
                trainingCorpusClasses_ls.append('spam')

print('{} ham training files read'.format(trainingCorpusClasses_ls.count('ham')))
print('{} spam training files read'.format(trainingCorpusClasses_ls.count('spam')))

In [ ]:
testCorpusDocuments_ls = []
testCorpusClasses_ls = []

# First collect the ham documents:
print("Reading ham test files...")

for (path, dirs, files) in os.walk('./data/testData/ham/'):
    
    for file in files:
        if file[0] == '.':  # Don't process hidden files
            continue
        
        with open(os.path.join(path, file), 'rb') as fileIn:
            docText = fileIn.read()
            docText = docText.decode('utf-8', 'ignore')   # decoding the utf-8
            
            testCorpusDocuments_ls.append(docText)
            testCorpusClasses_ls.append('ham')
            
# Next, collect the spam documents:
print("Reading spam test files...")

for (path, dirs, files) in os.walk('./data/testData/spam/'):
    
    for file in files:
        if file[0] == '.':  # Don't process hidden files
            pass
        else:
            with open(os.path.join(path, file), 'rb') as fileIn:
                docText = fileIn.read()
                docText = docText.decode('utf-8', 'ignore')   # decoding the utf-8

                testCorpusDocuments_ls.append(docText)
                testCorpusClasses_ls.append('spam')

print('{} ham test files read'.format(testCorpusClasses_ls.count('ham')))
print('{} spam test files read'.format(testCorpusClasses_ls.count('spam')))

## Tokenising the dataset

As before, we will use the `tokenise_email_document` to perform the tokenisation:

In [ ]:
trainingTokenisedDocuments_ls = [tokenise_email_document(doc_txt) for doc_txt in trainingCorpusDocuments_ls]

## Building an inverse document frequency index

Having imported and tokenised the data, we now need to build the inverse document frequency index. Recall that the definition of inverse document frequency (idf) for some term is:

$$\text{idf}(term)=\log_e\left(\frac{\textrm{total number of documents}}{\textrm{number of documents containing }term}\right)$$

As with the term frequency index we built in Notebook 22.3, we can build a *pandas* Series which contains the inverse document frequency values for all the terms in the training set.

First, create the term index of all the terms which appear in the training set:

In [ ]:
trainingTermIndex_ls = build_term_index(trainingTokenisedDocuments_ls)

Next, we want a Series which represents how many documents each term appears in (the 'number of documents containing `term`' in the definition of `idf`). We will start with a Series whose index is the terms which appear in the collection, and which has zero for each document frequency:

In [ ]:
documentFrequencyIndex_ss = pd.Series(0, index=trainingTermIndex_ls)

We can populate the Series with the document frequency count for each term:

In [ ]:
for tokenisedDoc_ls in trainingTokenisedDocuments_ls:
    for term in set(tokenisedDoc_ls):
        documentFrequencyIndex_ss[term] += 1

So, for example, to find out how many documents the term *bill* appears in, use:

In [ ]:
documentFrequencyIndex_ss['bill']

We can now create the idf index by dividing the number of documents in the training corpus by the document frequency, and using `np.log` to find the log of the values in the Series:

In [ ]:
idfIndex_ss = pd.Series(len(trainingCorpusDocuments_ls),  # Put the number of documents as
                        index=trainingTermIndex_ls)       # each value

idfIndex_ss = np.log(idfIndex_ss / documentFrequencyIndex_ss)  # Divide by the document 
                                                               # frequency and take the log

We can now compare the impact that different terms will have. Comparing the inverse document frequency values of the terms *the* and *bill*:

In [ ]:
print(idfIndex_ss['the'])
print(idfIndex_ss['bill'])

shows that each occurence of *bill* will be much more heavily weighted than each occurrence of *the*.

## Reducing the training data size

As before, we will quickly run into memory problems if we try to create a DataFrame containing the complete set of training documents, so as before we will only use the most common terms:

In [ ]:
termFrequencyIndex_ss = pd.Series(0, index=idfIndex_ss.index)

for tokenisedDoc_ls in trainingTokenisedDocuments_ls:
    for token in tokenisedDoc_ls:
        termFrequencyIndex_ss[token] += 1

termFrequencyIndex_ss.sort_values(ascending=False, inplace=True)
        
termFrequencyIndex_ss.head()

Again, take the 200 most common terms and create an index containing only those terms:

In [ ]:
shortTermIndex = termFrequencyIndex_ss.index[:200]

shortTermIndex

And use the `reindex` method to reduce the size of `idfIndex_ss`:

In [ ]:
idfIndex_ss = idfIndex_ss.reindex(shortTermIndex)

idfIndex_ss

## Building and training the classifier

We can now build our set of training vectors. Previously, we used the term frequency for each term in the sentence. In this case, we multiply the term frequency by the inverse document frequency value for that term (to give tf.idf):

In [ ]:
trainingTfIdfVectors_ls = [build_tf_vector(tokenisedDoc_ls, shortTermIndex) * idfIndex_ss
                           for tokenisedDoc_ls in trainingTokenisedDocuments_ls]

In [ ]:
trainingData_df = pd.DataFrame(trainingTfIdfVectors_ls)

trainingData_df

And as before, use this DataFrame and the training classes to build a *k*-NN classifier. Again, we will use *k*=3.

In [ ]:
spamFilter3_knn = KNeighborsClassifier(n_neighbors=3, metric='cosine', algorithm='brute')

In [ ]:
spamFilter3_knn.fit(trainingData_df,
                    trainingCorpusClasses_ls)

## Using the classifier to classify test data

To classify the test data, we need the tf.idf vector for each vector in the test set. First tokenise the test data:

In [ ]:
testTokenisedDocuments_ls = [tokenise_email_document(doc_txt) for doc_txt in testCorpusDocuments_ls]

and convert to tf.idf vectors:

In [ ]:
testTfIdfVectors_ls = [build_tf_vector(tokenisedDoc_ls, shortTermIndex) * idfIndex_ss
                       for tokenisedDoc_ls in testTokenisedDocuments_ls]

In [ ]:
testData_df = pd.DataFrame(testTfIdfVectors_ls)

testData_df

Finally, apply the classifier to the test data:

In [ ]:
results_df = pd.DataFrame({'predicted':spamFilter3_knn.predict(testData_df),
                           'actual':testCorpusClasses_ls})

results_df

## Evaluating the filter

Again, we can use the `pd.crosstab` function to present the results in a more readable way:

In [ ]:
tabulatedResults_df = pd.crosstab(results_df.predicted, results_df.actual, margins=True)

tabulatedResults_df

We can now print the results, and give an overall percentage accuracy (total number of emails that were correctly classified into *ham* or *spam*):

In [ ]:
print('Ham correctly classified as ham: {}/{}'.format(tabulatedResults_df['ham']['ham'],
                                                      tabulatedResults_df['ham']['All']))

print('Ham incorrectly classified as spam: {}/{}'.format(tabulatedResults_df['ham']['spam'],
                                                         tabulatedResults_df['ham']['All']))

print('Spam incorrectly classified as ham: {}/{}'.format(tabulatedResults_df['spam']['ham'],
                                                         tabulatedResults_df['spam']['All']))

print('Spam correctly classified as spam: {}/{}'.format(tabulatedResults_df['spam']['spam'],
                                                        tabulatedResults_df['spam']['All']))

print('Overall system accuracy: {:.1%}'.format((tabulatedResults_df['ham']['ham'] + 
                                                tabulatedResults_df['spam']['spam']) / 
                                                     tabulatedResults_df['All']['All']))

This is an (even) stronger result than our previous attempt.

The key message to take away here is that we have managed to greatly improve the behaviour of our data application by considering the nature of the data (natural language documents), choosing an appropriate similarity measure (cosine similarity) and using knowledge of the dataset to improve the way the data is processed (tf.idf measures) in a way that is appropriate for the particular application.

## What next?
If you are working through this Notebook as part of an inline exercise, return to the module materials now.

If you are working through this set of Notebooks as a whole, you've completed the Part 22 Notebooks.